In [1]:
%run -i ../python/common.py
UC_SKIPTERMS=True
%run -i ../python/ln_preamble.py

# SLS Lecture 14 : Assembly using the OS

- Review of Processes
- Review of Systems Calls
- Process Address Spaces 
  - Dynamic Memory : Adding and Growing our heap
- I/O - Assembly "Hello World"

In [2]:
# setup for sumit examples
appdir=os.getenv('HOME')
appdir=appdir + "/syscalls"
#print(movdir)
output=runTermCmd("[[ -d " + appdir + " ]] &&  rm -rf "+ appdir + 
             ";mkdir " + appdir + 
             ";cp ../src/Makefile ../src/setup.gdb ../src/usesum5.s ../src/sumit6.s ../src/usesumheap.s ../src/hello.s ../src/brk.s ../src/100.nums " + appdir)

display(Markdown('''
- create a directory `mkdir syscalls; cd syscalls`
- copy examples
- add a `Makefile` to automate assembling and linking
    - we are going run the commands by hand this time to highlight the details
- add our `setup.gdb` to make working in gdb easier
- normally you would want to track everything in git
'''))
TermShellCmd("ls " + appdir)


- create a directory `mkdir syscalls; cd syscalls`
- copy examples
- add a `Makefile` to automate assembling and linking
    - we are going run the commands by hand this time to highlight the details
- add our `setup.gdb` to make working in gdb easier
- normally you would want to track everything in git


Output(layout=Layout(border='1px solid black', height='100%', overflow_y='scroll'))

## Review of Processes

### Processes and the Kernel
<center>
<img src="../images/UnixRunning.png">
    </center>

<center>
    <img src="../images/Processes/Processes.004.png">
</center>

<center>
    <img src="../images/Processes/Processes.005.png">
</center>

## Review of System Calls - From Lecture 08

In [3]:
display(Markdown(htmlFig("../images/kerneldef.png")))

<!-- produced by: 
htmlFig("[{'src': '../images/kerneldef.png'}]", 
        id="", 
        align="center", 
        width="100%",
        margin="auto auto auto auto",
        caption="", 
        captionalign="left")
-->
<table align="center" width="100%" cellpadding="0" cellspacing="0" border="0" style="border-collapse: collapse; margin: auto auto auto auto">
    <tr style="padding: 0; margin: 0;"> 
        <td colspan="1" width="100%" style="padding: 0; margin: 0 0 0 0; background-color:inherit;">
            <div style="padding: 0; margin: 0 0 0 0;">
              <figure style="padding: 0; margin: 0 0 0 0; width:100%;">
                   <img src="../images/kerneldef.png" width="100%" style="padding: 0; margin: 0;">
                </figure>
            </div>
        </td>
    </tr>
</table>


Remember this...

**The "Kernel" -- Unique to Every OS**

1. Bootstraps the HW and has direct access to all of it
1. Bottom layer that enables other programs to run
2. <font style="color:red;"> **A unique collection of functions that programs can invoke** </font>

Not useful on its own only useful and accessed by running other programs.

<center>
<img src="../images/Processes/Processes.051.png">
</center>

### Intel `syscall`

On Intel the instruction is `syscall`

<img src="../images/syscallmp.png">



### The OS System Calls

Each OS Kernel provides a set of calls that a process can invoke using the `syscall` instruction on an Intel based computer

The Linux Kernel supports a very large number of system calls each is identified by a unique number that must be placed in `RAX` prior to executing the `syscall` instruction.  Additional arguments are passed in by setting other registers.  

With each version of the Kernel the table of calls changes.  Here is one site that provides a list

### LINUX SYSTEM CALLS

- reading some man pages `man syscalls` and `man syscall` we find that
  - to exit we must place `60` in `rax`
  - and that the value we want to set as our exit status code in `rdi`
- each system call will accept arguments in various registers 
- NOTE: **On INTEL 64bit processors `rcx` will be overwritten by the syscall instruction**
- On INTEL 64bit processors Linux system calls will return a value back in `rax` and possibly `rdx`
  - `rax` and possibly `rdx` will be overwritten by a system call

In [4]:
TermShellCmd("man syscalls | head -20", markdown=False)
TermShellCmd("man syscall | head -20", markdown=False)

Output(layout=Layout(border='1px solid black', height='100%', overflow_y='scroll'))

In [5]:
display(IFrame("https://filippo.io/linux-syscall-table/", height=600, width="100%"))

In [6]:
display(Markdown(FileCodeBox(
    file="../src/exit_bb_bb.s", 
    lang="gas", 
    title="",
    h="100%", 
    w="100%"
)))


<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` gas
	.intel_syntax noprefix
	
	.text
	.global _start
_start:      	
	mov     rax, 60    # Linux exit system call number is 60       
	mov     rdi, 0     # rdi is return value 0 success

	syscall                         	

```


</div>


## Process Address Spaces

### Address Space organizes Memory of a Process

<center>
    <img src="../images/ProcessContexts.png">
</center>

**Remember**

1. The binaries we create using the assembler and linker are Executables

2. When we "run" our executables via the shell it creates a new process context and our binary is loaded as the initial memory "image".

3. The memory and register values of the process are unique to each process and change as the instructions of our binary "execute"

### Assembly Programming as Process Address Space Image creation

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.003.png">
</center>

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.004.png">
</center>

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.005.png">
</center>

### The Details

### Process Address Space 

The Memory of a Process is organized as an (Virtual) Address Space 

1. Each possible location of a byte is identified by a unique address (number) 
2. To associate a Region (range of continues addresses) with actual memory requires a "mapping"
    - mappings associate a Region with Memory and a possible source of values
    - mappings can restrict what kind of access can be made to region 
        1. fetch for execute: x
        2. read: r
        3. write: w

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-001.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-002.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-003.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-004.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-005.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-006.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-007.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-008.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-009.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-010.png">
</center>

<center>
    <img src="../images/ExecAddrSpace/ExecAddrSpace-011.png">
</center>

<hr>

## Summary of how we "loading" a process address from a binary 

<hr>

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.012.png">
</center>

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.016.png">
</center>

### Exploring the Address Space of a process with an example

- Let's explore the address space of a running program 

In [7]:
display(Markdown(FileCodeBox(
    file="../src/usesum5.s", 
    lang="gas", 
    title="<b>CODE: asm - usesum5.s",
    h="100%", 
    w="107em"
)))

<b>CODE: asm - usesum5.s
<div style="width:107em; height:100%; font-size:inherit; overflow: auto;" >


``` gas
	.intel_syntax noprefix
	.data	
	.comm A_SUM, 8, 8
	.comm A_LEN, 8, 8
	.comm A, 8*1024, 8	

	.comm B_SUM, 8, 8
	.comm B_LEN, 8, 8
	.comm B, 8*1024, 8	

	.comm C_SUM, 8, 8
	.comm C_LEN, 8, 8
	.comm C, 8*1024, 8	

	.text
	.global _start 
_start:
	mov rbx, QWORD PTR [A_LEN]
	mov rcx, OFFSET A
	call sumIt
	mov QWORD PTR [A_SUM], rax

	mov rbx, QWORD PTR [B_LEN]
	mov rcx, OFFSET B
	call sumIt
	mov QWORD PTR [B_SUM], rax

	mov rbx, QWORD PTR [C_LEN]
	mov rcx, OFFSET C
	call sumIt
	mov QWORD PTR [C_SUM], rax
	int3

```


</div>


In [8]:
display(Markdown(FileCodeBox(
    file="../src/sumit6.s", 
    lang="gas", 
    title="<b>CODE: asm - sumit6.s",
    h="100%", 
    w="107em"
)))

<b>CODE: asm - sumit6.s
<div style="width:107em; height:100%; font-size:inherit; overflow: auto;" >


``` gas
	.intel_syntax noprefix

	.section .text

	# tell linker that sumIt symbol can be referenced in other files	
	.global sumIt	  

	# code to sum data in array who's address is in rcx
	# we assume we were started with call so return address on stack
	# we assume rbx has length  rbx -> len
	# and that we will leave final sum in rax
sumIt:	                         # label that marks where this code begins
	xor  rax, rax            # rax -> sum : sum = 0
	push rdi                 # spill current value of rdi before we use it
	xor  rdi, rdi            # rdi -> i : i = 0
	
	# code to sum data in array who's address is in rcx
	# we assume we were started with call so return address on stack
	# we assume rbx has length  rbx -> len
	# and that we will leave final sum in rax
loop_start:                                 
	cmp  rbx, rdi                       # rbx - rdi
	jz   loop_done                      # if above is zero (they are equal) jump
	add  rax, QWORD PTR [rcx + rdi * 8] # add the i'th value to the sum
	inc  rdi                            # i=i+1
	jmp  loop_start                     # go back to the start of the loop
loop_done:
	pop  rdi                            # restore rdi back to its original value
	ret                                 # use return to pop value off the stack
	                                    # and jump to that location


```


</div>


In [9]:
TermShellCmd("make usesum6", cwd=appdir, prompt='')

Output(layout=Layout(border='1px solid black', height='100%', overflow_y='scroll'))

In [10]:
showDT()

<b>Debug</b>

```
b _start
run 
info proc
```

```
 cat /proc/<pid>/maps
```

Use gdb to explore what's there and compare with binary binary file 

In [11]:
TermShellCmd("man proc | head -30", markdown=False)

Output(layout=Layout(border='1px solid black', height='100%', overflow_y='scroll'))

In [12]:
TermShellCmd("man proc | grep -A40 /maps | head -41", markdown=False)

Output(layout=Layout(border='1px solid black', height='100%', overflow_y='scroll'))

### Let's another section to control how region is mapped

Read-only data section `.rodata`

```nasm
         .section .rodata

 str1:
         .string "Hello World\n"
```

What would happen if we put all our data in the `.rodata` section... try it and find out

## Ok lets get some more memory -- And and Grow our Heap

The `brk` system call - move "break pointer"

- initially the data segment of your process will end at some location based on what data sections you defined
  - this location is called the program break pointer 
  - the location where valid data memory ends and the un-allocated virtual address space
- with `brk` you can set the end to a value bigger than it starting location
- you can also shrink it break pointer back 

This area is called the HEAP

1. call `brk` with argument of 0 and Linux kernel returns current break address
2. call `brk` "with a reasonable address" and the kernel will allocated or de-allocated memory for us

- in reality once we move to C we never directly call "brk" rather we will use a library routines called : `malloc` and `free` which will call `brk` for us.


In [13]:
TermShellCmd("man brk | head -37", markdown=False)

Output(layout=Layout(border='1px solid black', height='100%', overflow_y='scroll'))

In [14]:
display(IFrame("https://filippo.io/linux-syscall-table/", height=600, width="100%"))

**Lets add this code**

What does it do?

```gas
    .data
NEW_MEM_PTR:
    .quad 0

    .text
    
    mov rdi, 0
    mov rax, 12
    syscall 
    mov QWORD PTR [NEW_MEM_PTR], rax
    
    mov rdi, rax
    add rdi, 1024 * 8
    mov rax, 12
    syscall
```

In [15]:
display(Markdown(FileCodeBox(
    file="../src/usesumheap.s", 
    lang="gas", 
    title="<b>CODE: asm - usesumheap.s",
    h="100%", 
    w="107em"
)))

<b>CODE: asm - usesumheap.s
<div style="width:107em; height:100%; font-size:inherit; overflow: auto;" >


``` gas
	.intel_syntax noprefix
	.section .data	
	.comm A_SUM, 8, 8
	.comm A_LEN, 8, 8
	.comm A, 8*1024, 8	

	.comm B_SUM, 8, 8
	.comm B_LEN, 8, 8
	.comm B, 8*1024, 8	

	.comm C_SUM, 8, 8
	.comm C_LEN, 8, 8
	.comm C, 8*1024, 8	

	# Added space heap memory array
	.comm D_SUM, 8, 8
	.comm D_LEN, 8, 8
	.comm D_PTR, 8, 8

	.section .rodata
mystr:	
	.string "Hello World!!!\n"
	
	.section .text
	.global _start 
_start:
	# get current break pointer
	mov rdi, 0                 # pass 0 to brk (invalid request)
	mov rax, 12                # brk syscall number 12
	syscall                    # call brk with 0
	mov QWORD PTR [D_PTR], rax # set D_PTR to start of end of break memory
	mov rdi, rax               # mov current end of break into rax
	
	add rdi, 1024 * 8          # add request num bytes to end break
	mov rax, 12                # brk syscall number 12
	syscall                    # call brk with break end + request

	mov rbx, QWORD PTR [D_LEN] 
	mov rcx, QWORD PTR [D_PTR]
	call sumIt
	mov QWORD PTR [D_SUM], rax

	mov rbx, QWORD PTR [A_LEN]
	mov rcx, OFFSET A
	call sumIt
	mov QWORD PTR [A_SUM], rax

	mov rbx, QWORD PTR [B_LEN]
	mov rcx, OFFSET B
	call sumIt
	mov QWORD PTR [B_SUM], rax

	mov rbx, QWORD PTR [C_LEN]
	mov rcx, OFFSET C
	call sumIt
	mov QWORD PTR [C_SUM], rax

	mov rdi, rax
	mov rax, 60
	syscall
	


```


</div>


**What does the regions maps look like now?**

In [16]:
showDT()
display(Markdown(FileCodeBox(
    file="../src/usesumheap.gdb", 
    lang="gas", 
    title="<b>usesumheap gdb",
    h="100%", 
    w="107em"
)))

<b>usesumheap gdb
<div style="width:107em; height:100%; font-size:inherit; overflow: auto;" >


``` gas
b _start
run
display /1gx &D_PTR
x/10gd (void *)D_PTR
si
si
si
si
x/10gd (void *)D_PTR
si
si
si
si
x/10gd (void *)D_PTR
restore 100.bin binary <addr>
x/10gd (void *)D_PTR
set *((long long *)&D_LEN)=100
disass
break *addr
c
si
display /1gd D_SUM



```


</div>


## I/O

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.017.png">
</center>

<center>
    <img src="../images/ASSEMBLY-AddressSpaceandIO/ASSEMBLY-AddressSpaceandIO.018.png">
</center>

### Review from Lecture 3

<center>
    <img src="../images/Processes/Processes.006.png">
</center>

<center>
    <img src="../images/Processes/Processes.007.png">
</center>

<center>
    <img src="../images/Processes/Processes.008.png">
</center>

<center>
    <img src="../images/Processes/Processes.009.png">
</center>

<center>
    <img src="../images/Processes/Processes.010.png">
</center>

<center>
    <img src="../images/Processes/Processes.011.png">
</center>

<center>
    <img src="../images/Processes/Processes.012.png">
</center>

<center>
    <img src="../images/Processes/Processes.013.png">
</center>

<center>
    <img src="../images/Processes/Processes.015.png">
</center>

### Finally "Hello World in Assembly"

In [17]:
display(Markdown(FileCodeBox(
    file="../src/hello.s", 
    lang="gas", 
    title="<b>CODE: asm - hello.S",
    h="100%", 
    w="107em"
)))

<b>CODE: asm - hello.S
<div style="width:107em; height:100%; font-size:inherit; overflow: auto;" >


``` gas
	.intel_syntax noprefix
	.section .rodata
str:
	.string "Hello World\n"

	.section .text
	.global _start
_start:
	mov rax, 1
	mov rdi, 1
	mov rsi, OFFSET str
	mov rdx, 12
	syscall

	mov rax, 60
	mov rdi, 3
	syscall
	

```


</div>


In [18]:
TermShellCmd("make hello", cwd=appdir, prompt='')

Output(layout=Layout(border='1px solid black', height='100%', overflow_y='scroll'))

#### At long last we have a program that we don't need the debugger for!

In [19]:
TermShellCmd("./hello; echo $?", cwd=appdir)

Output(layout=Layout(border='1px solid black', height='100%', overflow_y='scroll'))

### There are many more calls for doing I/O

But we can now 
1. transfer bytes from our address space to a "file" - write system call
2. transfer bytes from a "file" to our address space  - read system call
3. connect (and possibly create) files to our address space for reading and writing - open system call
4. disconnect a file from our address space - close system call


In [20]:
display(IFrame("https://filippo.io/linux-syscall-table/", height=600, width="100%"))

### Exercises

Change the `usesum` program to:
- exit properly 
- allocate memory for the data
- read the data in from standard in
- write the binary result to standard out
  - use a shell command like `od -l -A10` to convert the output to ascii
- open the binary file of numbers and read them in
- can you figure out how to get the command line arguments?
- Write a new program that uses the random instruction to create a data file
